# Collecting the Stop Words

In [1]:
with open("StopWords_Auditor.txt") as my_text:
    a = my_text.read().lower().split()
    uni_a = set(a) # getting unique words

In [2]:
uni_a

{'coopers',
 'deloitte',
 'ernst',
 'kpmg',
 'pricewaterhouse',
 'pricewaterhousecoopers',
 'touche',
 'young'}

In [3]:
with open("StopWords_GenericLong.txt") as my_txt:
    b = my_txt.read().lower().split()
    uni_b = set(b)

In [4]:
with open("StopWords_DatesandNumbers.txt") as my_text:
    c = my_text.read().lower().split()
    uni_c = set(c)

In [5]:
with open("StopWords_Generic.txt") as my_txt:
    d = my_txt.read().lower().split()
    uni_d = set(d)

In [6]:
with open("StopWords_Geographic.txt") as my_txt:
    e = my_txt.read().lower().split()
    uni_e = set(e)

In [7]:
with open("StopWords_Names.txt") as my_txt:
    f = my_txt.read().lower().split()
    uni_f = set(f)

In [8]:
with open("StopWords_Currencies.txt") as my_text:
    g = my_text.read().lower().split()
    uni_g = set(g)

In [9]:
all_uni_words = set()

In [10]:
all_uni_words.update(uni_a) # updating all the unique words
all_uni_words.update(uni_b)
all_uni_words.update(uni_c)
all_uni_words.update(uni_d)
all_uni_words.update(uni_e)
all_uni_words.update(uni_f)
all_uni_words.update(uni_g)

In [11]:
len(all_uni_words) # this consists all the words across all documents

12840

In [12]:
full_vocab = dict()
i = 0

for word in all_uni_words:
    full_vocab[word] = i
    i += 1

In [13]:
all_stop_words = [""]*len(full_vocab)

In [14]:
# converting the dictionary to list
for word in full_vocab:
    word_index = full_vocab[word]
    all_stop_words[word_index] += word

In [15]:
len(all_stop_words)

12840

# Text Analysis

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [17]:
pip install textblob

Note: you may need to restart the kernel to use updated packages.


In [18]:
import requests
import bs4

In [19]:
from bs4 import BeautifulSoup

In [20]:
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [21]:
import re

In [22]:
from textblob import TextBlob
from nltk.corpus import stopwords

In [23]:
Positive_Score = []
Negative_score = []
Polarity_Score = []
Subjectivity_Score = []
URL = []
Average_sentence_length = []
Word_count = []
Complex_words_count = []
Percentage_of_complex_words = []
Fog_Index = []
Personal_pronoun_counts = []

stop_words = set(stopwords.words('english'))

# adding our customized stop words
custom_stop_words = all_stop_words
stop_words.update(custom_stop_words)

def analyze_sentiment(tweet):
    analysis = TextBlob(tweet) #creating a TextBlob object from the tweet parameter
    #extract the polarity and subjectivity scores from the TextBlob object using its sentiment property
    polarity = analysis.sentiment.polarity 
    subjectivity = analysis.sentiment.subjectivity
    return polarity, subjectivity

df = pd.read_excel('C:\\Users\\User\\Downloads\\Input.xlsx', usecols=[1])

for index, row in df.iterrows():
    url = row[0]
    response = requests.get(url)

    soup = bs4.BeautifulSoup(response.content, "lxml")
    text = soup.getText(strip=True)
    
    #cleaning content by importing the "regular expression" library"
    #import re # regular expression
    text = re.sub(r'\[\d+\]',"",text)
    text = re.sub(r'\[\w+\]',"",text)
    text = re.sub('[0-9]+',"", text)

    sentences = sent_tokenize(text)
    sentiment_scores = [analyze_sentiment(sentence) for sentence in sentences if sentence not in stop_words] # stop_words
    
    sentences = nltk.sent_tokenize(text) # Tokenize the text into sentences

    # Get the total number of words in all the sentences
    total_words = sum(len(nltk.word_tokenize(sentence)) for sentence in sentences)
    
    total_sentences = len(sentences) #the total number of sentences

    # Calculating the average sentence length
    avg_sentence_length = total_words / total_sentences

    # Get the total number of complex words
    complex_words = []
    for sentence in sentences:
        words = nltk.word_tokenize(sentence)
        for word in words:
            if len(word) > 2 and nltk.pos_tag([word])[0][1] in ['JJ', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 
                                                                'RB', 'RBR', 'RBS']:
                complex_words.append(word)
    total_complex_words = len(complex_words)

    # Calculating the percentage of complex words
    percent_complex_words = (total_complex_words / total_words) * 100

    # Get the total number of unique words
    unique_words = set(nltk.word_tokenize(text))

    # Get the total number of words
    total_words = len(nltk.word_tokenize(text))
    
    # Define the list of personal pronouns to search for
    personal_pronouns = ["I", "we", "my", "us", "ours"]

    # Tokenize the text into words
    words = nltk.word_tokenize(text.lower())

    # Count the number of occurrences of each personal pronoun
    personal_pronoun_counts = [words.count(pronoun) for pronoun in personal_pronouns]
    personal_pronoun_counts = sum(personal_pronoun_counts) # summing the numbers of the list

    # Count the number of positive and negative polarity scores
    polarity_scores = [score[0] for score in sentiment_scores]
    positive_scores = sum(1 for score in polarity_scores if score > 0)
    negative_scores = sum(1 for score in polarity_scores if score < 0)

    # Calculate the average polarity and subjectivity scores
    avg_polarity = sum(polarity_scores) / len(polarity_scores)
    avg_subjectivity = sum([score[1] for score in sentiment_scores]) / len(sentiment_scores)

    Positive_Score.append(positive_scores)
    Negative_score.append(negative_scores)
    Polarity_Score.append(avg_polarity)
    Subjectivity_Score.append(avg_subjectivity)
    URL.append(url)
    Average_sentence_length.append(avg_sentence_length)
    Percentage_of_complex_words.append(percent_complex_words)
    Fog_Index.append(0.4 * (avg_sentence_length + percent_complex_words))
    Word_count.append(total_words)
    Complex_words_count.append(total_complex_words)
    Personal_pronoun_counts += [personal_pronoun_counts]

# Calculating Average Number of words per sentence

In [24]:
Average_Number_of_words_per_sentence = []

def analyze_sentiment(tweet):
    analysis = TextBlob(tweet) #creating a TextBlob object from the tweet parameter
    #extract the polarity and subjectivity scores from the TextBlob object using its sentiment property
    polarity = analysis.sentiment.polarity 
    subjectivity = analysis.sentiment.subjectivity
    return polarity, subjectivity

df = pd.read_excel('C:\\Users\\User\\Downloads\\Input.xlsx', usecols=[1])

for index, row in df.iterrows():
    url = row[0]
    response = requests.get(url)

    soup = bs4.BeautifulSoup(response.content, "lxml")
    text = soup.getText(strip=True)

    text = re.sub(r'\[\d+\]',"",text)
    text = re.sub(r'\[\w+\]',"",text)
    text = re.sub('[0-9]+',"", text)
    
    sentences = sent_tokenize(text)
    sentiment_scores = [analyze_sentiment(sentence) for sentence in sentences if sentence not in stop_words] # stop_words
    
    sentences = nltk.sent_tokenize(text) # Tokenize the text into sentences

    # Getting the total number of words in all the sentences
    total_words = sum(len(nltk.word_tokenize(sentence)) for sentence in sentences)
    
    total_sentences = len(sentences) #the total number of sentences
    
    Average_Number_of_words_per_sentence += [total_words / total_sentences]

# Calculating Average Word Length

In [65]:

def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    polarity = analysis.sentiment.polarity 
    subjectivity = analysis.sentiment.subjectivity
    return polarity, subjectivity

def calculate_avg_word_length(text):
    words = text.split()
    total_length = sum(len(word) for word in words)
    return total_length / len(words)

df = pd.read_excel('C:\\Users\\User\\Downloads\\Input.xlsx', usecols=[1])

Average_Word_Length = []

for index, row in df.iterrows():
    url = row[0]
    response = requests.get(url)

    soup = bs4.BeautifulSoup(response.content, "lxml")
    text = soup.getText(strip=True)

    text = re.sub(r'\[\d+\]',"",text)
    text = re.sub(r'\[\w+\]',"",text)
    text = re.sub('[0-9]+',"", text)

    sentences = sent_tokenize(text)
    sentiment_scores = [analyze_sentiment(sentence) for sentence in sentences if sentence not in stop_words]

    total_words = sum(len(nltk.word_tokenize(sentence)) for sentence in sentences for sentence in sentences)

    total_sentences = len(sentences)

    Average_Number_of_words_per_sentence = total_words / total_sentences
    Average_Word_Length.append(calculate_avg_word_length(text))

In [66]:
# create a new list with the rounded elements from the original Average_Word_Length list:
Average_Word_len = []
for num in Average_Word_Length:
    Average_Word_len.append(round(num, 0))

# Creating a DataFrame

In [109]:
data = pd.read_excel("C:\\Users\\User\\Downloads\\Input.xlsx")

In [110]:
index  = data["URL_ID"] # appending the Url_id directly from the excel sheet's first column

In [111]:
df = pd.DataFrame({"URL ID":index,'URL': URL, 'Positive Score': Positive_Score, 'Negative score': Negative_score, 
                   'Polarity Score': Polarity_Score, 'Subjectivity Score': Subjectivity_Score,
                  "Average sentence length":Average_sentence_length, "Percentage of complex words":Percentage_of_complex_words,
                  "Fog Index":Fog_Index,
                   "Average Number of words per sentence":Average_Number_of_words_per_sentence,
                   "Complex words count":Complex_words_count,
                   "Word count":Word_count,
                  "Personal Pronouns":Personal_pronoun_counts,
                  "Average Word length":Average_Word_len})

In [107]:
df.columns = df.columns.str.upper()

In [1]:
#df.tail()

# Extracting as Excel File

In [114]:
# Extracting to Excel
#df.to_excel('Output Data.xlsx', index=False)